# 📌 9.8 파이썬 추천 시스템 패키지 - Surprise

패키지 설치!

In [2]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163698 sha256=a30af89df2900c3271d04b25d4174aa31346dd7962b871daf8e9696d105857e7
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


## Surprise를 이용한 추천 시스템 구축

패키지 불러오기!

In [3]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [4]:
data = Dataset.load_builtin('ml-100k')
#수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


위에서 다운받은 무비렌즈 데이터 세트가 /root/.surprise_data/ml-100k에 저장된 것을 확인 가능!  
코랩에선 위에 루트의 링크 누르면 파일로 확인 가능함!  
여기서 사용할 ml-100k 데이터는 분리문자가 앞선 데이터들과는 달리 콤마가 아닌 탭(\t)인 것을 유의해야함.  
그리고 서프라이즈 패키지는 앞선 다른 예제들과는 달리 자체적으로 로우 데이터를 칼럼 레벨로 변경해주기 때문에, 기존에 있던 로우 레벨의 데이터의 아이템 아이디를 칼럼명으로 변환하는 과정을 생략해야함!

In [5]:
#SVD 객체 생성
algo = SVD(random_state=0)
#학습 수행
algo.fit(trainset)

test() -> 사용자-아이템 평점 데이터 세트 전체에 대해 추천 예측  
predict() -> 개별 사용자와 영화에 대한 추천 평점 반환

### test 객체

In [6]:
predictions = algo.test(testset)
print('prediction type :',type(predictions), 'size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'> size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1807542478219157, details={'was_impossible': False})]

위의 결과 해석!
- test의 결과는 list 객체이며, 크기가 입력된 데이터 세트 크기와 같은 25000이다.
- 개별사용자 아이디(uid), 영화(=아이템) 아이디(iid), 실제 평점(r_ui), 추천 예측 평점(est)를 튜플 형태로 가짐
- details -> 추천 예측 할 수 없는 경우 로그용으로 데이터 남기는 용도! was_impossible -> True이면 예측값 생성 불가라는 것!

In [8]:
#3개 prediction에서 개별 아이디, 아이템 아이디, 예측평점을 추출해보자!
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

### predict 객체

predict는 개별 사용자의 아이템에 대한 추천 평점을 예측, 따라서 인자로 개별 사용자 아이디, 아이템 아이디를 입력해야함! 그러면 그에 따른 예측 평점을 보여줌.

In [9]:
#사용자 아이디, 아이템 아이디는 문자열로 입력해야 함.
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


-> 196번 고객은, 302번 아이템에 대해 평점을 남기지 않았지만, 예측하자면 4.49의 평점을 남길 것이고, 평점 예측값을 생성 가능하다!

In [10]:
#위에서 만든 테스트 예측 가지고 rmse 계산해서 정확도 평가하기!
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

## Surprise 주요 모듈 소개

9.6 에서 사용한 데이터들 로딩!

In [11]:
#디렉터리 설정
from google.colab import drive
drive.mount('/content/gdrive/')

path = "/content/gdrive/My Drive/EuronData/"

Mounted at /content/gdrive/


### OS 파일 데이터 로딩

In [13]:
import pandas as pd

ratings = pd.read_csv(path+'ml-latest-small/ratings.csv')
#ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성. -> 파일 확인해보면 생성되어있음!
ratings.to_csv(path + 'ml-latest-small/ratings_noh.csv', index=False, header=False)

In [16]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
#user item rating timestamp로 총 4개의 칼럼으로 구성되어있고, 분리문자는 콤마, 최소평점은 0.5, 최대는 5라고 명시함!
#로딩시 ratings_noh.csv 파일에서 앞의 3개 칼럼만 로딩되고 timestamp는 제외됨!
data=Dataset.load_from_file(path + 'ml-latest-small/ratings_noh.csv',reader = reader)

In [17]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

#수행시마다 동일한 결과 도출을 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)

#학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### 판다스 데이터 프레임 사용

In [18]:
import pandas as pd
from surprise import Reader, Dataset

#데이터를 데이터프레임으로 로딩!
ratings = pd.read_csv(path +'ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

#ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

## 교차 검증과 하이처 파라미터 튜닝

In [19]:
from surprise.model_selection import cross_validate

#판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv(path + 'ml-latest-small/ratings.csv') #reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8763  0.8796  0.8693  0.8720  0.8727  0.8740  0.0036  
MAE (testset)     0.6711  0.6776  0.6692  0.6690  0.6708  0.6716  0.0031  
Fit time          2.31    2.18    2.33    4.33    1.81    2.60    0.89    
Test time         0.13    0.29    0.13    0.38    0.23    0.23    0.10    


{'test_rmse': array([0.87628265, 0.8796379 , 0.86933308, 0.87201295, 0.87266801]),
 'test_mae': array([0.67108594, 0.67759036, 0.66924026, 0.66902667, 0.67081837]),
 'fit_time': (2.314889907836914,
  2.1830952167510986,
  2.334709405899048,
  4.3340795040130615,
  1.8098113536834717),
 'test_time': (0.12882304191589355,
  0.29024314880371094,
  0.12540245056152344,
  0.378673791885376,
  0.232696533203125)}

cross_validate()를 통해 폴드별 성능 평가 수치, 평균 성능 평가 수치 확인 가능.  

In [20]:
from surprise.model_selection import GridSearchCV

#최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

#CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mae로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

#최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8765119428687583
{'n_epochs': 20, 'n_factors': 50}


-> 'n_epochs': 20, 'n_factors': 50 일 때 가장 최적으로 당시 rmse가 0.8765임을 확인 가능

## Surprise를 이용한 개인화 영화 추천 시스템 구축

In [21]:
#다음 코드는 train_test_split()으로 분리되지 않는 데이터 세트에 fit()을 호출해 오류가 발생합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'n_users'

DatasetAutoFolds 클래스로 데이터 전체를 학습 데이터로 이용  


In [22]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
#DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성.
data_folds = DatasetAutoFolds(ratings_file=path + 'ml-latest-small/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함.
trainset = data_folds.build_full_trainset()

위에서 생성한 학습 데이터 기반으로 SVD 학습 진행

In [23]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

특정 사용자(userid = 9인 사용자)를 지정해서 해당 사용자가 영화 42에 대한 평점을 아직 매기지 않았는지 확인 후 평점 예측 진행

In [24]:
#영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv(path + 'ml-latest-small/movies.csv')

#userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인.
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


영화 아이디 42에 대한 정보(제목, 장르) 확인 가능.

In [25]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


9번 사용자는 42번 영화에 대해 평점을 남기지 않았고 아마도 평점을 3.13으로 남길 것이고 그 평점을 예측 가능하다.

get_unseen_surprise() 함수를 생성해서, 인자로 들어온 사용자 아이디의 사용자가 평점을 남기지 않은 영화 return.

In [26]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()

    # 모든 영화들의 movieId를 리스트로 생성.
    total_movies = movies['movieId'].tolist()

    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), '전체 영화수:',len(total_movies))

    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10) 함수를 생성,  
-> 추천 영화 대상으로 추천 알고리즘 객체의 predict() 이용, 저장된 리스트 내부의 prediction을 리스트로 저장, 그 후 예측 평점이 높은 top-n개의 영화 아이디, 영화 제목, 예측 평점을 반환.

In [27]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    #알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

    #predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    #[Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,]
    #이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    #sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est

    #sortkey_est() 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]

    #top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122657
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
